In [1]:
raw_data_dir = '../data/raw/'
interim_data_dir = '../data/interim'

In [2]:
import requests
import json
import pandas as pd

url= "https://crashviewer.nhtsa.dot.gov/CrashAPI"
#/crashes/GetCrashesByLocation?fromCaseYear=2014&toCaseYear=2015&state=1&county=1&format=json

fromCaseYear = "2010"
toCaseYear = "2020"
state = "6"
qurl = f"{url}/crashes/GetCrashesByLocation?fromCaseYear={fromCaseYear}&toCaseYear={toCaseYear}&state={state}&county=73&format=json"

cali=requests.get(qurl).json()
data = (json.dumps(cali,indent=4,sort_keys=True))

CITYNAME = []
COUNTY =  []
COUNTYNAME = []
CaseYear = []
FATALS = []
LATITUDE = []
LONGITUD = []
STATE = []
STATENAME = []
ST_CASE = []
TOTALVEHICLES = []
TWAY_ID = []
TWAY_ID2 = []
VE_FORMS = []
# Loop through the list of cities and perform a request for data on each
for COUNTYNAME in COUNTYNAME:
    response = requests.get(qurl).json()
    CITYNAME.append(response[0]['CITYNAME'])


In [3]:
pd.DataFrame(cali['Results'][0])

,CITY,CITYNAME,COUNTY,COUNTYNAME,CaseYear,FATALS,LATITUDE,LONGITUD,STATE,STATENAME,ST_CASE,TOTALVEHICLES,TWAY_ID,TWAY_ID2,VE_FORMS
0,3260,SAN DIEGO,73,SAN DIEGO (73),2014,1,32.76605000,-117.150869440,6,California,60005,2,I-8,,2
1,0,NOT APPLICABLE,73,SAN DIEGO (73),2013,1,33.37609444,-117.257936110,6,California,60006,2,CR-ALTURAS RD,,1
2,3260,SAN DIEGO,73,SAN DIEGO (73),2013,1,32.80097222,-117.142119440,6,California,60007,4,MURRAY RIDGE RD,,1
3,3260,SAN DIEGO,73,SAN DIEGO (73),2018,1,32.57529722,-117.081208330,6,California,60007,1,I-5,,1
4,1080,EL CAJON,73,SAN DIEGO (73),2014,1,32.79522222,-116.935727780,6,California,60008,2,JAMACHA ROAD,,2
5,3260,SAN DIEGO,73,SAN DIEGO (73),2015,1,32.74262500,-117.254688890,6,California,60009,1,BACON STREET,CORONADO AVENUE,1
6,3260,SAN DIEGO,73,SAN DIEGO (73),2012,1,32.79469444,-117.171125000,6,California,60009,1,GENESEE AVE,,1
7,3260,SAN DIEGO,73,SAN DIEGO (73),2012,1,32.75406667,-117.130180560,6,California,60010,1,HOWARD AVE,30TH ST,1
8,1200,ESCONDIDO,73,SAN DIEGO (73),2016,1,33.11413056,-117.104897220,6,California,60011,1,HALE ST,AUTOPARK WY,1
9,1820,LA MESA,73,SAN DIEGO (73),2016,1,32.75183056,-117.014827780,6,California,60014,1,SR-94 RAMP,,1


In [110]:
fromCaseYear = "2010"
toCaseYear = "2020"
state = "6"

qurl = f"{url}/crashes/GetCaseDetails?stateCase=63474&caseYear=2016&state=6&format=json"
data = requests.get(qurl).json()
print(data)
case = data['Results'][0][0]['CrashResultSet']

{'Count': 1, 'Message': 'Results returned successfully', 'Results': [[{'CrashResultSet': {'ARR_HOUR': '99', 'ARR_HOURNAME': 'Unknown EMS Scene Arrival Hour', 'ARR_MIN': '99', 'ARR_MINNAME': 'Unknown EMS Scene Arrival Minutes', 'CEvents': [{'AOI1': '81', 'AOI1NAME': 'Right', 'AOI2': '61', 'AOI2NAME': 'Left', 'CaseYear': '2016', 'EVENTNUM': '1', 'SOE': '12', 'SOENAME': 'Motor Vehicle In-Transport ', 'STATE': '6', 'STATENAME': 'California', 'ST_CASE': '63474', 'VNUMBER1': '1', 'VNUMBER2': '2'}, {'AOI1': '55', 'AOI1NAME': 'Non-Harmful Event', 'AOI2': '55', 'AOI2NAME': 'Non-Harmful Event', 'CaseYear': '2016', 'EVENTNUM': '2', 'SOE': '63', 'SOENAME': 'Ran Off Roadway - Right', 'STATE': '6', 'STATENAME': 'California', 'ST_CASE': '63474', 'VNUMBER1': '1', 'VNUMBER2': '5555'}, {'AOI1': '98', 'AOI1NAME': 'Not Reported', 'AOI2': '77', 'AOI2NAME': 'Not a Motor Vehicle', 'CaseYear': '2016', 'EVENTNUM': '3', 'SOE': '33', 'SOENAME': 'Curb', 'STATE': '6', 'STATENAME': 'California', 'ST_CASE': '63474',

In [ ]:
def extract_people(v):
    for p in v['Persons']:
        yield {
            'Speed Limite Exceeded': v['SPEEDRELNAME'],
            'Speed limite': v['TRAV_SP'],
            'Vin Number': v['VINNAME'],
            'Traveled Speed Veh': v['VSPD_LIM'],
            'Make': v['MAKENAME'],
            'Make/Model': v['MAK_MODNAME'],
            'Model': v['MODELNAME'],
            
            "Age": p['AGE'],
            "Age Name": p['AGENAME'],
            "County": p['COUNTYNAME'],
            "Death DANAME": p['DEATH_DANAME'],
            "DOA Name": p['DOANAME'],
            "Injury": p['INJ_SEVNAME'],
            "Race": p['RACENAME'],
            "Age": p["AGENAME"],
            "Death": p["DOANAME"],
            "Road_Type": p["ROAD_FNCNAME"],
            "Sex": p["SEXNAME"],
            "Make": p["MAKENAME"],
        }


        
def get_people(case):

    hour = case['HOUR']
    minute = case['MINUTE']
    time = f"{hour}:{minute}"
        
    accident_info = {
        'Case Number': case['ST_CASE'],
        "Description of Veh Coll": case['CF2NAME'], 
        "Day of Week": case['DAY_WEEKNAME'],
        "Drunk Driver": case['DRUNK_DR'],
        "Year": case['CaseYear'],
        "Month": case['MonthName'],
        "Hour": hour,
        "Time of Accident": time,
    }

    vehicles = case['Vehicles']
    
    people = [{**accident_info, **p} for v in vehicles for p in extract_people(v)]
    
    return pd.DataFrame(people)


def get_events(case):
    c_events = [{
        'Case Number': case['ST_CASE'],
# In a traffic accident AOI is Area of Impact. The spot the two cars collided is measured 
# to a fixed object, usually the curb, so it can be reconstructed later.
        'Area of Impact': e['AOI1NAME'],
# standard of evidence
# https://safety.fhwa.dot.gov/rsdp/cdip_rpti.aspx
        'Standard of Evenidence': e['SOENAME'],
        'Event Number': e['EVENTNUM'],
        'Vehicle 1': e['VNUMBER1'],
        'Vehicle 2': e['VNUMBER2'],
    } for e in case['CEvents']]
    
    return pd.DataFrame(c_events)




In [108]:
get_people(case)

,Age,Age Name,Case Number,County,DOA Name,Day of Week,Death,Death DANAME,Description of Veh Coll,Drunk Driver,...,Month,Race,Road_Type,Sex,Speed Limite Exceeded,Speed limite,Time of Accident,Traveled Speed Veh,Vin Number,Year
0,22 Years,22 Years,60005,SAN DIEGO (73),Died at Scene,Thursday,Died at Scene,9,"Motor Vehicle struck by falling cargo,or somet...",0,...,January,Black,Urban-Principal Arterial-Interstate,Male,"Yes, Exceeded Speed Limit",95,16:10,65,JS1GN7EA6821,2014
1,53 Years,53 Years,60005,SAN DIEGO (73),Not Applicable,Thursday,Not Applicable,Not Applicable (Non-Fatal),"Motor Vehicle struck by falling cargo,or somet...",0,...,January,Not a Fatality (not Applicable),Urban-Principal Arterial-Interstate,Female,No,45,16:10,65,KMHWF25S23A7,2014


In [106]:
get_events(case)

,Area of Impact,Case Number,Event Number,Standard of Evenidence,Vehicle 1,Vehicle 2
0,Non-Collision,60005,1,Rollover/Overturn,1,9999
1,Other Objects Set-In-Motion,60005,2,Motor Vehicle In-Transport Strikes or is Struc...,1,2
2,Not Reported,60005,3,Motor Vehicle In-Transport,1,2
3,Non-Collision,60005,4,Fire/Explosion,1,9999
4,Non-Collision,60005,5,Fire/Explosion,2,9999
